# Create a Target Forecast
* **About 40 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# Recover variables stored by other notebooks
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Create Prophet and DeepAR+ Campaign

In [4]:
# Prophet
prophet_forecastName = project+'_prophet_algo_forecast' + target_suffix + suffix
prophet_create_forecast_response=forecast.create_forecast(
    ForecastName=prophet_forecastName,
    PredictorArn=target_prophet_predictorArn)
target_prophet_forecast_arn = prophet_create_forecast_response['ForecastArn']

In [5]:
forecast.describe_forecast(ForecastArn = target_prophet_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggle_prophet_algo_forecast_target35988',
 'ForecastName': 'WalmartKaggle_prophet_algo_forecast_target35988',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggle_prophet_algo_1_target35988',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/WalmartKaggleDSG35988',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 2, 3, 53, 356000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 2, 3, 53, 356000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'ffd677fc-c17a-4fb4-baa5-c59d820b534f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 02:03:52 GMT',
   'x-amzn-requestid': 'ffd677fc-c17a-4fb4-baa5-c59d820b534f',
   'content-length': '517',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [6]:
# DeepAR+
deeparp_forecastName = project+'_deeparp_algo_forecast' + target_suffix + suffix
deeparp_create_forecast_response=forecast.create_forecast(
    ForecastName=deeparp_forecastName,
    PredictorArn=target_deepar_predictorArn)
target_deeparp_forecast_arn = deeparp_create_forecast_response['ForecastArn']

In [7]:
forecast.describe_forecast(ForecastArn = target_deeparp_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggle_deeparp_algo_forecast_target35988',
 'ForecastName': 'WalmartKaggle_deeparp_algo_forecast_target35988',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggle_deeparp_algo_135988',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/WalmartKaggleDSG35988',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 2, 3, 53, 692000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 2, 3, 53, 692000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '720a8b96-3fad-46d1-b41e-faf02c86fbdd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 02:03:53 GMT',
   'x-amzn-requestid': '720a8b96-3fad-46d1-b41e-faf02c86fbdd',
   'content-length': '510',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [8]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast(ForecastArn= target_prophet_forecast_arn)['Status']
    createDeeparpStatus = forecast.describe_forecast(ForecastArn= target_deeparp_forecast_arn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PR

## Upload forecast results to S3

In [9]:
target_prophet_path = "s3://" + bucket_name + "/" + bucket_folder + "/prophet_" + target_suffix + suffix + "/" 
target_prophet_job_name = "ProphetExport1" + target_suffix + suffix
create_forecast_export_job_prophet_response = forecast.create_forecast_export_job(
    ForecastExportJobName = target_prophet_job_name,
    ForecastArn = target_prophet_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": target_prophet_path,
            "RoleArn": role_arn
        }
    })


In [10]:
TargetForecastProphetExportJobArn = create_forecast_export_job_prophet_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = TargetForecastProphetExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:us-east-2:057716757052:forecast-export-job/WalmartKaggle_prophet_algo_forecast_target35988/ProphetExport1_target35988',
 'ForecastExportJobName': 'ProphetExport1_target35988',
 'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggle_prophet_algo_forecast_target35988',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-us-east-2-057716757052/WalmartKaggle/prophet__target35988',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRoleWalmart35988'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 2, 32, 58, 812000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 2, 32, 58, 812000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '8f997be5-8a22-4ae7-8d7b-947972d380b9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 02:32:58 GMT',
   'x-amzn-requestid': '8f997be5-8a22-4ae7-8d7b-947972d380b9'

In [11]:
target_deeparp_path = "s3://" + bucket_name + "/" + bucket_folder + "/deeparp_" + target_suffix + suffix + "/" 
target_deeparp_job_name = "DeepARPExport1" + target_suffix + suffix
create_forecast_export_job_deepar_response = forecast.create_forecast_export_job(
    ForecastExportJobName = target_deeparp_job_name,
    ForecastArn = target_deeparp_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": target_deeparp_path,
            "RoleArn": role_arn
        }
    })

In [12]:
TargetForecastDeeparExportJobArn = create_forecast_export_job_deepar_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = TargetForecastDeeparExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:us-east-2:057716757052:forecast-export-job/WalmartKaggle_deeparp_algo_forecast_target35988/DeepARPExport1_target35988',
 'ForecastExportJobName': 'DeepARPExport1_target35988',
 'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/WalmartKaggle_deeparp_algo_forecast_target35988',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-us-east-2-057716757052/WalmartKaggle/deeparp__target35988',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRoleWalmart35988'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 5, 2, 32, 59, 41000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 5, 2, 32, 59, 41000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '65b70c92-0a3c-4547-ba4a-7e8bb3b40680',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 05 Apr 2020 02:32:58 GMT',
   'x-amzn-requestid': '65b70c92-0a3c-4547-ba4a-7e8bb3b40680',


In [13]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= TargetForecastProphetExportJobArn)['Status']
    createDeeparpStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= TargetForecastDeeparExportJobArn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 43.9 ms, sys: 4.39 ms, total: 48.3 ms
Wall time: 5min


In [14]:
%store target_prophet_forecast_arn
%store target_deeparp_forecast_arn
%store TargetForecastProphetExportJobArn
%store TargetForecastDeeparExportJobArn



Stored 'target_prophet_forecast_arn' (str)
Stored 'target_deeparp_forecast_arn' (str)
Stored 'TargetForecastProphetExportJobArn' (str)
Stored 'TargetForecastDeeparExportJobArn' (str)
